In [58]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn import svm
import numpy as np


In [10]:
def f(x):
    if type(x) == str:
        return "".join(x.split(" ")[:2])
    else:
        return np.nan

In [11]:
def g(x):
    return str(x.split(" ")[-1])

def h(x):
    return str(x.split(" ")[-2])

In [12]:
import numpy as np
def split_col(col):
    if not type(col) == float:
        return float(col.split("m")[0])
    else:
        return np.NaN

In [44]:
train = pd.read_csv("train.csv", dtype=str)
test = pd.read_csv("test.csv", dtype=str)
train.shape, test.shape

((246, 11), (62, 10))

In [45]:
# fix an issue at data
test.loc[test[test.period == 'USA'].index.values[0], "lived_in"] = "USA"
test.loc[test[test.period == 'USA'].index.values[0], "period"] = np.nan

In [46]:
#train = train.dropna()

train["short_period"]=train["period"].apply(f)

#train["is_herb"] = train["diet"].apply(is_herb)
#train["is_omni"] = train["diet"].apply(is_omni)
#train["is_carnivorous"] = train["diet"].apply(is_carnivorous)

train["length"] = train["length"].apply(split_col)

In [47]:
print(f"test shape: {test.shape}")
#test = test.dropna()
#print(f"test shape after dropna: {test.shape}")

test["short_period"]=test["period"].apply(f)

#test["is_herb"] = test["diet"].apply(is_herb)
#test["is_omni"] = test["diet"].apply(is_omni)
#test["is_carnivorous"] = test["diet"].apply(is_carnivorous)

test["length"] = test["length"].apply(split_col)

test shape: (62, 10)


In [48]:
feature_names = ["diet","short_period","lived_in","taxonomy"]

In [49]:
trainX = train[feature_names].copy(deep=True)
testX = test[feature_names].copy(deep=True)

In [50]:
trainX["data"] = "train"
testX["data"] = "test"

In [51]:
trainX

,diet,short_period,lived_in,taxonomy,data
0,carnivorous,LateTriassic,United Kingdom,Dinosauria,train
1,herbivorous/omnivorous,LateTriassic,Argentina,Dinosauria Saurischia Sauropodomorpha Prosauro...,train
2,herbivorous,LateCretaceous,China,Dinosauria Ornithischia Genasauria Cerapoda Or...,train
3,herbivorous,LateCretaceous,USA,Dinosauria Saurischia Sauropodomorpha Sauropod...,train
4,herbivorous,EarlyCretaceous,United Kingdom,Dinosauria Ornithischia Genasauria Cerapoda Or...,train
...,...,...,...,...,...
241,herbivorous,EarlyCretaceous,Mongolia,Dinosauria Ornithischia Genasauria Thyreophora...,train
242,herbivorous,EarlyCretaceous,Australia,Dinosauria Ornithischia Genasauria Cerapoda Or...,train
243,carnivorous,EarlyCretaceous,China,Dinosauria Saurischia Theropoda Neotheropoda T...,train
244,carnivorous,LateCretaceous,Argentina,Dinosauria Saurischia Theropoda Neotheropoda C...,train


In [52]:
testX

,diet,short_period,lived_in,taxonomy,data
0,herbivorous,LateCretaceous,Romania,Dinosauria Saurischia Sauropodomorpha Sauropod...,test
1,herbivorous,LateJurassic,USA,Dinosauria Saurischia Sauropodomorpha Sauropod...,test
2,carnivorous,LateCretaceous,USA,Dinosauria Saurischia Theropoda Neotheropoda T...,test
3,carnivorous,EarlyCretaceous,USA,Dinosauria Saurischia Theropoda Neotheropoda T...,test
4,omnivorous,NaN,USA,Dinosauria Saurischia Theropoda Neotheropoda T...,test
...,...,...,...,...,...
57,herbivorous,MidJurassic,China,Dinosauria Saurischia Sauropodomorpha Sauropod...,test
58,herbivorous,LateCretaceous,Mongolia,Dinosauria Ornithischia Genasauria Thyreophora...,test
59,carnivorous,LateCretaceous,Morocco,Dinosauria Saurischia Theropoda Neotheropoda T...,test
60,herbivorous,LateCretaceous,Canada,Dinosauria Ornithischia Genasauria Cerapoda Ma...,test


In [53]:
train_test_X = pd.concat([trainX, testX], ignore_index=True)
ex_train_test_X_dummies = pd.get_dummies(train_test_X)

transformer = make_column_transformer(
    (OneHotEncoder(sparse=False), feature_names),
    remainder='passthrough'
)
transformed = transformer.fit_transform(train_test_X)
train_test_X_dummies = pd.DataFrame(
    transformed,
    columns=transformer.get_feature_names_out()
)

train_X=train_test_X_dummies.iloc[:trainX.shape[0]]
test_X=train_test_X_dummies.iloc[trainX.shape[0]:]
train_y = train["type"].values

C:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [54]:
train_X

,onehotencoder__diet_carnivorous,onehotencoder__diet_herbivorous,onehotencoder__diet_herbivorous/omnivorous,onehotencoder__diet_omnivorous,onehotencoder__diet_unknown,onehotencoder__short_period_EarlyCretaceous,onehotencoder__short_period_EarlyJurassic,onehotencoder__short_period_LateCretaceous,onehotencoder__short_period_LateJurassic,onehotencoder__short_period_LateTriassic,...,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Maniraptoriformes Ornithomimosauria,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Maniraptoriformes Ornithomimosauria Ornithomimidae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea Tyrannosauridae Albertosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea Tyrannosauridae Tyrannosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Megalosauridae Eustreptospondylinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Megalosauridae Megalosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Spinosauridae Baryonychinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Spinosauridae Spinosaurinae,remainder__data
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
242,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
243,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
244,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train


In [55]:
test_X

,onehotencoder__diet_carnivorous,onehotencoder__diet_herbivorous,onehotencoder__diet_herbivorous/omnivorous,onehotencoder__diet_omnivorous,onehotencoder__diet_unknown,onehotencoder__short_period_EarlyCretaceous,onehotencoder__short_period_EarlyJurassic,onehotencoder__short_period_LateCretaceous,onehotencoder__short_period_LateJurassic,onehotencoder__short_period_LateTriassic,...,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Maniraptoriformes Ornithomimosauria,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Maniraptoriformes Ornithomimosauria Ornithomimidae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea Tyrannosauridae Albertosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea Tyrannosauridae Tyrannosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Megalosauridae Eustreptospondylinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Megalosauridae Megalosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Spinosauridae Baryonychinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Spinosauridae Spinosaurinae,remainder__data
246,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
247,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
248,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,test
249,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
250,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
304,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
305,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test


In [27]:
test_X

,onehotencoder__diet_carnivorous,onehotencoder__diet_herbivorous,onehotencoder__diet_herbivorous/omnivorous,onehotencoder__diet_omnivorous,onehotencoder__short_period_EarlyCretaceous,onehotencoder__short_period_EarlyJurassic,onehotencoder__short_period_LateCretaceous,onehotencoder__short_period_LateJurassic,onehotencoder__short_period_LateTriassic,onehotencoder__short_period_MidJurassic,...,onehotencoder__taxonomy_before_end_Stegosauria,onehotencoder__taxonomy_before_end_Stegosauridae,onehotencoder__taxonomy_before_end_Tetanurae,onehotencoder__taxonomy_before_end_Therizinosauroidea,onehotencoder__taxonomy_before_end_Theropoda,onehotencoder__taxonomy_before_end_Titanosauria,onehotencoder__taxonomy_before_end_Titanosauriformes,onehotencoder__taxonomy_before_end_Tyrannoraptora,onehotencoder__taxonomy_before_end_Tyrannosauridae,remainder__data


In [56]:
train_test_X_dummies

,onehotencoder__diet_carnivorous,onehotencoder__diet_herbivorous,onehotencoder__diet_herbivorous/omnivorous,onehotencoder__diet_omnivorous,onehotencoder__diet_unknown,onehotencoder__short_period_EarlyCretaceous,onehotencoder__short_period_EarlyJurassic,onehotencoder__short_period_LateCretaceous,onehotencoder__short_period_LateJurassic,onehotencoder__short_period_LateTriassic,...,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Maniraptoriformes Ornithomimosauria,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Maniraptoriformes Ornithomimosauria Ornithomimidae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea Tyrannosauridae Albertosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Avetheropoda Coelurosauria Tyrannoraptora Tyrannosauroidea Tyrannosauridae Tyrannosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Megalosauridae Eustreptospondylinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Megalosauridae Megalosaurinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Spinosauridae Baryonychinae,onehotencoder__taxonomy_Dinosauria Saurischia Theropoda Neotheropoda Tetanurae Spinosauroidea Spinosauridae Spinosaurinae,remainder__data
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
304,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
305,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test
306,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test


In [61]:
train_X=train_X.drop("remainder__data", axis=1)
test_X = test_X.drop("remainder__data", axis=1)

In [62]:
from sklearn.model_selection import train_test_split
linears = []
rbfs = []

for random_state in range(1000):
    print(f"{100*(random_state/1000)}%")
    X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.20, random_state=random_state)

    linear_svm_classifier = svm.SVC(kernel="linear") # linear kernel
    rbf_svm_classifier = svm.SVC(kernel="rbf") # rbf kernel


    # fit classifiers && train models
    linear_svm_classifier.fit(X_train, y_train)
    rbf_svm_classifier.fit(X_train, y_train)

    # make predictions using trained models
    linear_y_pred = linear_svm_classifier.predict(X_test)
    rbf_y_pred = rbf_svm_classifier.predict(X_test)

    # evaluate scores


    linears.append(metrics.accuracy_score(y_test, linear_y_pred))
    rbfs.append(metrics.accuracy_score(y_test, rbf_y_pred))



print(f"Max linear accuracy on : {linears.index(max(linears))} with {max(linears)} (rbf: {rbfs[linears.index(max(linears))]})")
print(f"Max rbf accuracy on : {rbfs.index(max(rbfs))} with {max(rbfs)}")

0.0%
0.1%
0.2%
0.3%
0.4%
0.5%
0.6%
0.7000000000000001%
0.8%
0.8999999999999999%
1.0%
1.0999999999999999%
1.2%
1.3%
1.4000000000000001%
1.5%
1.6%
1.7000000000000002%
1.7999999999999998%
1.9%
2.0%
2.1%
2.1999999999999997%
2.3%
2.4%
2.5%
2.6%
2.7%
2.8000000000000003%
2.9000000000000004%
3.0%
3.1%
3.2%
3.3000000000000003%
3.4000000000000004%
3.5000000000000004%
3.5999999999999996%
3.6999999999999997%
3.8%
3.9%
4.0%
4.1000000000000005%
4.2%
4.3%
4.3999999999999995%
4.5%
4.6%
4.7%
4.8%
4.9%
5.0%
5.1%
5.2%
5.3%
5.4%
5.5%
5.6000000000000005%
5.7%
5.800000000000001%
5.8999999999999995%
6.0%
6.1%
6.2%
6.3%
6.4%
6.5%
6.6000000000000005%
6.7%
6.800000000000001%
6.9%
7.000000000000001%
7.1%
7.199999999999999%
7.3%
7.3999999999999995%
7.5%
7.6%
7.7%
7.8%
7.9%
8.0%
8.1%
8.200000000000001%
8.3%
8.4%
8.5%
8.6%
8.7%
8.799999999999999%
8.9%
9.0%
9.1%
9.2%
9.3%
9.4%
9.5%
9.6%
9.700000000000001%
9.8%
9.9%
10.0%
10.100000000000001%
10.2%
10.299999999999999%
10.4%
10.5%
10.6%
10.7%
10.8%
10.9%
11.0%
11.1%
11

In [77]:
skf = StratifiedKFold(n_splits=5)
for x,y in skf.split(train_X, train_y):
    print(train_y[x].shape)
    print("-")
    print(train_y[y].shape)

(196,)
-
(50,)
(197,)
-
(49,)
(197,)
-
(49,)
(197,)
-
(49,)
(197,)
-
(49,)


In [82]:
# Load the dataset
# Initialize stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5)  #the number of folds is 5

# Perform stratified k-fold cross-validation
rbf_accuracy_scores = []
linear_accuracy_scores = []
for train_index, test_index in skf.split(train_X, train_y):
    X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]

    linear_svm_classifier = svm.SVC(kernel="linear") # linear kernel
    rbf_svm_classifier = svm.SVC(kernel="rbf") # rbf kernel


    # fit classifiers && train models
    linear_svm_classifier.fit(X_train, y_train)
    rbf_svm_classifier.fit(X_train, y_train)

    # make predictions using trained models
    linear_y_pred = linear_svm_classifier.predict(X_test)
    rbf_y_pred = rbf_svm_classifier.predict(X_test)

    # evaluate scores


    linear_accuracy_scores.append(metrics.accuracy_score(y_test, linear_y_pred))
    rbf_accuracy_scores.append(metrics.accuracy_score(y_test, rbf_y_pred))


# Calculate the mean accuracy
mean_accuracy = np.mean(linear_accuracy_scores)
rbf_mean_accuracy = np.mean(rbf_accuracy_scores)
print("Mean accuracy:", mean_accuracy)
print("Mean accuracy rbf:", rbf_mean_accuracy)


Mean accuracy: 0.8333061224489796
Mean accuracy rbf: 0.7844897959183673


In [84]:
from sklearn.metrics import f1_score

f1_scores = []
k = 10
skf = StratifiedKFold(n_splits=k)

# Perform k-fold stratified cross-validation
for train_index, test_index in skf.split(train_X, train_y):
    X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]

    linear_svm_classifier = svm.SVC(kernel="linear") # linear kernel
    # fit classifiers && train models
    linear_svm_classifier.fit(X_train, y_train)
    # make predictions using trained models
    linear_y_pred = linear_svm_classifier.predict(X_test)

    # Compute the weighted-average F1-score for this fold
    fold_f1_score = f1_score(y_test, linear_y_pred, average='weighted')
    f1_scores.append(fold_f1_score)

# Calculate the mean F1-score across all folds
mean_weighted_f1_score = np.mean(f1_scores)
print("Mean weighted-average F1-score across", k, "folds:", mean_weighted_f1_score)

Mean weighted-average F1-score across 10 folds: 0.8421161967661966


In [168]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.20, random_state=66)
from sklearn import svm

linear_svm_classifier = svm.SVC(kernel="linear") # linear kernel
rbf_svm_classifier = svm.SVC(kernel="rbf") # rbf kernel


# fit classifiers && train models
linear_svm_classifier.fit(X_train, y_train)
rbf_svm_classifier.fit(X_train, y_train)

# make predictions using trained models
linear_y_pred = linear_svm_classifier.predict(X_test)
rbf_y_pred = rbf_svm_classifier.predict(X_test)

# evaluate scores
from sklearn import metrics

print(f"accuracy linear: {metrics.accuracy_score(y_test, linear_y_pred)}")
print(f"accuracy rbf: {metrics.accuracy_score(y_test, rbf_y_pred)}")

accuracy linear: 0.9782608695652174
accuracy rbf: 0.9347826086956522


In [175]:
test_X["type_linear"] = linear_svm_classifier.predict(test_X[train_X.columns])
test_X["type_rbf"] = linear_svm_classifier.predict(test_X[train_X.columns])


In [181]:
results=test_X.reset_index().reset_index()
results.rename(columns={"level_0":"id"}, inplace=True)
results["id"] = results["id"] + 1
linear_svm_results = results[["id","type_linear"]]
linear_svm_results.rename(columns={"type_linear":"type"}, inplace=True)
rbf_svm_results = results[["id","type_rbf"]]
rbf_svm_results.rename(columns={"type_rbf":"type"}, inplace=True)

C:\Users\alini\AppData\Local\Temp\ipykernel_25668\1550707187.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_svm_results.rename(columns={"type_linear":"type"}, inplace=True)
C:\Users\alini\AppData\Local\Temp\ipykernel_25668\1550707187.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rbf_svm_results.rename(columns={"type_rbf":"type"}, inplace=True)


In [182]:
rbf_svm_results

,id,type
0,1,sauropod
1,2,euornithopod
2,3,large theropod
3,4,large theropod
4,5,sauropod
...,...,...
57,58,sauropod
58,59,armoured dinosaur
59,60,small theropod
60,61,ceratopsian


In [183]:
linear_svm_results

,id,type
0,1,sauropod
1,2,euornithopod
2,3,large theropod
3,4,large theropod
4,5,sauropod
...,...,...
57,58,sauropod
58,59,armoured dinosaur
59,60,small theropod
60,61,ceratopsian


In [193]:
(linear_svm_results == rbf_svm_results).mean().mean()

1.0

In [195]:
linear_svm_results.to_csv("submission_2_linear_svm.csv",index=False)
rbf_svm_results.to_csv("submission_2_rbf_svm.csv",index=False)

In [78]:
categories = ["diet", "period", "lived_in", "length", "taxonomy"]
train = train.astype("category", categories=categories)
test = test.astype("category", categories=categories)

TypeError: NDFrame.astype() got an unexpected keyword argument 'categories'

In [486]:
tX = testX.copy(deep=True)
for category in to_numeric_categories:
    tX[category].replace(category_dict[category][0], category_dict[category][1], inplace=True)

tX.dtypes

is_herb                  int64
is_omni                  int64
is_carnivorous           int64
short_period            object
taxonomy                object
lived_in                object
taxonomy_end            object
taxonomy_before_end     object
length                 float64
dtype: object

In [487]:
tX

,is_herb,is_omni,is_carnivorous,short_period,taxonomy,lived_in,taxonomy_end,taxonomy_before_end,length
0,1,0,0,1,2,27,2,2,5.0
1,1,0,0,4,Dinosauria Saurischia Sauropodomorpha Sauropod...,2,Camarasauridae,Camarasauromorpha,23.0
2,0,0,1,1,60,2,60,12,8.6
3,0,0,1,2,29,2,29,15,12.0
4,0,1,0,USA,Dinosauria Saurischia Theropoda Neotheropoda T...,NaN,Oviraptorosauria,38,3.0
...,...,...,...,...,...,...,...,...,...
57,1,0,0,3,11,1,11,11,5.0
58,1,0,0,1,27,4,27,23,6.0
59,0,0,1,1,16,14,16,14,8.1
60,1,0,0,1,9,6,9,9,6.0


In [488]:
def replace_unknown(x):

    if str(x).isdigit():
        return np.uint8(x)
    else:
        return np.NaN

for cg in to_numeric_categories:
    tX[cg] = tX[cg].apply(replace_unknown)

tX


,is_herb,is_omni,is_carnivorous,short_period,taxonomy,lived_in,taxonomy_end,taxonomy_before_end,length
0,1,0,0,1.0,2.0,27.0,2.0,2.0,5.0
1,1,0,0,4.0,NaN,2.0,NaN,NaN,23.0
2,0,0,1,1.0,60.0,2.0,60.0,12.0,8.6
3,0,0,1,2.0,29.0,2.0,29.0,15.0,12.0
4,0,1,0,NaN,NaN,NaN,NaN,38.0,3.0
...,...,...,...,...,...,...,...,...,...
57,1,0,0,3.0,11.0,1.0,11.0,11.0,5.0
58,1,0,0,1.0,27.0,4.0,27.0,23.0,6.0
59,0,0,1,1.0,16.0,14.0,16.0,14.0,8.1
60,1,0,0,1.0,9.0,6.0,9.0,9.0,6.0


In [489]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, [0])
    ])

In [490]:
preprocessor.fit_transform(tX)

ValueError: 'fill_value'=missing is invalid. Expected a numerical value when imputing numerical data

In [494]:
ttX=tX.dropna()

In [496]:
ttX

,is_herb,is_omni,is_carnivorous,short_period,taxonomy,lived_in,taxonomy_end,taxonomy_before_end,length
0,1,0,0,1.0,2.0,27.0,2.0,2.0,5.0
2,0,0,1,1.0,60.0,2.0,60.0,12.0,8.6
3,0,0,1,2.0,29.0,2.0,29.0,15.0,12.0
5,0,0,0,1.0,45.0,4.0,45.0,38.0,1.0
6,1,0,0,5.0,52.0,2.0,52.0,0.0,2.0
8,0,0,1,4.0,10.0,9.0,10.0,10.0,6.2
9,0,1,0,1.0,6.0,4.0,6.0,6.0,2.1
10,1,0,0,2.0,3.0,1.0,3.0,3.0,6.0
13,1,0,0,1.0,20.0,0.0,20.0,18.0,3.0
14,1,0,0,3.0,15.0,1.0,15.0,13.0,15.0


In [491]:
tX.dtypes

is_herb                  int64
is_omni                  int64
is_carnivorous           int64
short_period           float64
taxonomy               float64
lived_in               float64
taxonomy_end           float64
taxonomy_before_end    float64
length                 float64
dtype: object

In [497]:
types = linear_svm_classifier.predict(ttX)
ttX["type"] = types

C:\Users\alini\AppData\Local\Temp\ipykernel_3416\1121451900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ttX["type"] = types


In [498]:
ttX

,is_herb,is_omni,is_carnivorous,short_period,taxonomy,lived_in,taxonomy_end,taxonomy_before_end,length,type
0,1,0,0,1.0,2.0,27.0,2.0,2.0,5.0,1
2,0,0,1,1.0,60.0,2.0,60.0,12.0,8.6,0
3,0,0,1,2.0,29.0,2.0,29.0,15.0,12.0,2
5,0,0,0,1.0,45.0,4.0,45.0,38.0,1.0,4
6,1,0,0,5.0,52.0,2.0,52.0,0.0,2.0,3
8,0,0,1,4.0,10.0,9.0,10.0,10.0,6.2,2
9,0,1,0,1.0,6.0,4.0,6.0,6.0,2.1,4
10,1,0,0,2.0,3.0,1.0,3.0,3.0,6.0,1
13,1,0,0,1.0,20.0,0.0,20.0,18.0,3.0,1
14,1,0,0,3.0,15.0,1.0,15.0,13.0,15.0,0


In [500]:
ttX=ttX.reset_index()

In [503]:
ttX.rename(columns={"index":"id"}, inplace=True)

In [504]:
ttX

,id,is_herb,is_omni,is_carnivorous,short_period,taxonomy,lived_in,taxonomy_end,taxonomy_before_end,length,type
0,0,1,0,0,1.0,2.0,27.0,2.0,2.0,5.0,1
1,2,0,0,1,1.0,60.0,2.0,60.0,12.0,8.6,0
2,3,0,0,1,2.0,29.0,2.0,29.0,15.0,12.0,2
3,5,0,0,0,1.0,45.0,4.0,45.0,38.0,1.0,4
4,6,1,0,0,5.0,52.0,2.0,52.0,0.0,2.0,3
5,8,0,0,1,4.0,10.0,9.0,10.0,10.0,6.2,2
6,9,0,1,0,1.0,6.0,4.0,6.0,6.0,2.1,4
7,10,1,0,0,2.0,3.0,1.0,3.0,3.0,6.0,1
8,13,1,0,0,1.0,20.0,0.0,20.0,18.0,3.0,1
9,14,1,0,0,3.0,15.0,1.0,15.0,13.0,15.0,0


In [506]:
ttX.type.unique()

array([1, 0, 2, 4, 3], dtype=int64)

In [512]:
y.type.unique()

array(['sauropod', 'euornithopod', 'large theropod', 'armoured dinosaur',
       'small theropod', 'ceratopsian'], dtype=object)